# CBAM + CoordinateAttPlus (Adım-2)

## Amaç

Bu adımın amacı, **CBAM + Coordinate Attention** yapısını  
**YOLO / detection mimarilerinde stabil, güvenli ve patlamayan** bir attention bloğu
haline getirmektir.

Özellikle:
- küçük batch,
- AMP / FP16,
- erken eğitim evresi

gibi senaryolarda attention’ın **feature’ları aşırı bastırmasını** engellemek hedeflenmiştir.

---

## Ne eklendi? (Adım-2 ile gelen ana guardrail’ler)

### 1) Learnable Temperature (T) + Clamp
Channel Attention tarafındaki temperature (T):

- Öğrenilebilir veya sabit olabilir
- Her forward’da **[t_min, t_max]** aralığına clamp edilir
- Input ile **aynı device ve dtype**’a taşınır

**Amaç:**  
T’nin aşırı küçülüp attention’ı “hard gate”e çevirmesini  
ya da aşırı büyüyüp attention’ı etkisizleştirmesini önlemek.

---

### 2) Coordinate Attention Head’leri için güvenli init
Height ve Width attention head’leri:

- Küçük standart sapmalı normal dağılım ile başlatılır
- Bias’lar sıfırlanır

**Amaç:**  
Eğitimin ilk adımlarında maskelerin 0 veya 1’e yapışmasını engellemek.  
YOLO’da erken iterasyon stabilitesini artırmak.

---

### 3) Alpha ile “1’e karıştırma” (yumuşak attention)
Coordinate attention maskeleri **direkt uygulanmaz**.

Bunun yerine:
* scale = 1 + alpha * (attn - 1)


- `alpha → 0` : attention kapalıya yakın
- `alpha → 1` : attention tam uygulanır

**Amaç:**  
Attention’ın “var / yok” gibi sert davranması yerine,
etkisini **kontrollü ve kademeli** vermek.

---

### 4) Global Beta ile ek yumuşatma
Alpha’dan sonra tüm scale şu şekilde yumuşatılır:
* scale = 1 + beta * (scale - 1)


- `beta = 0` → attention tamamen kapalı
- `beta = 1` → scale aynen uygulanır

**Amaç:**  
Tüm attention bloğunun agresifliğini tek bir global düğme ile ayarlamak.

---

### 5) Scale Clamp (güvenlik bariyeri)
Son scale değeri:
* scale ∈ [scale_min, scale_max]
aralığına kilitlenir.

Örnek:
- en fazla %40 bastırma
- en fazla %60 güçlendirme

**Amaç:**  
Attention maskeleri ne yaparsa yapsın,
feature enerjisinin **kontrol dışına çıkmasını** engellemek.

---

### 6) Residual + Over-Suppression Monitor (EMA tabanlı)
Residual karışım şu formdadır:
* out = x + alpha_eff * (y - x)


Eğitim sırasında:
- giriş / çıkış enerji oranı ölçülür
- EMA ile yumuşatılır
- blok **fazla bastırıyorsa**, `alpha_eff` otomatik düşürülür

**Amaç:**  
Attention’ın feature’ları “öldürdüğü” durumlarda
bloğun kendi kendini yumuşatması.

---

## Hangi problemleri çözüyor?

- ❌ Erken eğitimde attention patlaması  
- ❌ Küçük batch’te over-suppression  
- ❌ AMP / FP16 device–dtype uyumsuzluğu  
- ❌ Attention’ın kontrolsüz agresifleşmesi  

---

## YOLO için neden güvenli?

- Giriş / çıkış boyutları korunur (drop-in block)
- AMP ve GPU uyumlu
- Attention hiçbir zaman sınırsız bastıramaz
- Residual enerji dengesini geri kazandırır
- Eğitim sırasında otomatik “rescue” vardır

---

## Tasarım Prensipleri (5 Madde)

1. **Attention asla sınırsız olmamalı**
2. **Her agresiflik için bir yumuşatma düğmesi olmalı**
3. **Residual enerji dengeleyici olarak kullanılmalı**
4. **Monitor sinyali öğrenmeden ayrı tutulmalı**
5. **Detection’da stabilite, ham güçten daha önemlidir**

---

## Tek Cümlelik Özet

Bu blok, CBAM + Coordinate Attention’ı  
**YOLO için enerji dostu, kontrollü ve kendi kendini dengeleyen**
bir attention yapısına dönüştürür.


-----
----
----
# Şimdi ise kod tarafından yorumlayarak gidelim.Kod tarafında bulunan yorumlara dikkat ediniz.Burda yer almayan açıklamalar için Yöntem-1 klasöründen ulaşabilirsiniz.Kod içerisinde nelerin bulunup bulunmadığı detaylı biçimde anlatılmıştır.


----
----
----

# 1) Büyük resim şeması ( Channel Attention )
**Bu modül her feature map için önce avg ve max ile kanalları özetliyor, sonra aynı MLP ile iki ayrı kanal skor seti çıkarıyor (avg tabanlı ve max tabanlı). Ardından ya basitçe topluyor ya da softmax router ile her sample için avg–max karışım ağırlığı öğrenip z’yi oluşturuyor. Sonra z’yi T ile sertlik ayarı yapıp gate’ten geçirerek kanal maskesi üretiyor ve en son beta ile 1’e yaklaştırıp feature’ı öldürmeyecek güvenli ölçekle x’i çarpıyor.**
```bash
x (B,C,H,W)
   |
   |-- AvgPool over (H,W) ---> avg_s (B,C,1,1) ----\
   |                                                \
   |-- MaxPool over (H,W) ---> max_s (B,C,1,1) ----->  MLP (aynı MLP)
                                                     /        \
                                             a=MLP(avg_s)    m=MLP(max_s)
                                             (B,C,1,1)       (B,C,1,1)
                                                     \        /
                                                      \      /
                                           fusion (sum veya softmax router)
                                                       |
                                                       v
                                                   z (B,C,1,1)
                                                       |
                                            divide by T (sertlik ayarı)
                                                       |
                                           gate (sigmoid/hardsigmoid)
                                                       |
                                                   ca (B,C,1,1)
                                                       |
                                       scale_ca = 1 + beta*(ca-1)
                                                       |
                                                       v
                                       y = x * scale_ca  (B,C,H,W)
```


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def softplus_inverse(y:torch.Tensor , eps:float = 1e-6)->torch.Tensor:
    return torch.log(torch.clamp(torch.exp(y) -1.0 , min=eps))
# Bu kod softplus fonksiyonunun tanımlanmış halidir.
# İleride öğrenilebilir temp tanımlarken kullanacağız.
# Bu haliyle aslında :: direkt y ve eps değerlerini yazarak kullanım sağlıyoruz.
# Kullanım ise  :: :: ::  t_inv = softplus_inverse(t0, eps=self.eps) 'dir

def _get_gate(gate: str):
    # Gate türünü string olarak alıyoruz (örn: "sigmoid", "hardsigmoid")
    # Ama dışarıdan büyük/küçük harf karışık gelebilir.
    # Bu yüzden normalize edip lower() yapıyoruz.
    g = gate.lower()
    # Klasik sigmoid:
    # - Çıkış aralığı (0, 1)
    # - Yumuşak ama pahalı (exp içerir)
    # - Attention maskelerinde sık kullanılır
    if g == "sigmoid":
        return torch.sigmoid
    # HardSigmoid:
    # - Sigmoid'in parçalı-lineer, daha ucuz versiyonu
    # - Mobil / YOLO / hız kritik senaryolarda tercih edilir
    # - Saturation daha kontrollüdür, FP16'da daha stabil olabilir
    if g == "hardsigmoid":
        return F.hardsigmoid
    # Buraya düşüyorsa:
    # - Kullanıcı desteklenmeyen bir gate ismi vermiştir
    # - Sessizce yanlış davranmak yerine FAIL FAST yapıyoruz
    # - Böylece config hataları erken yakalanır
    raise ValueError("gate 'sigmoid' veya 'hardsigmoid' olmalı.")


def _get_act(act: str):
    # Aktivasyon fonksiyonunu string ile seçiyoruz
    # Aynı şekilde case-insensitive olması için lower()
    a = act.lower()
    # ReLU:
    # - Basit, hızlı
    # - Negatifleri sıfırlar
    # - Attention MLP'lerinde hâlâ yaygın
    # inplace=True:
    # - Ekstra tensor allocation yok
    # - Bellek açısından daha verimli
    if a == "relu":
        return nn.ReLU(inplace=True)
    # SiLU (Swish):
    # - x * sigmoid(x)
    # - ReLU'dan daha yumuşak
    # - Gradient akışı daha stabil
    # - Modern CNN/YOLO varyantlarında daha çok tercih edilir
    if a == "silu":
        return nn.SiLU(inplace=True)
    # Desteklenmeyen aktivasyon verilirse:
    # - Sessiz fallback YOK
    # - Bilerek exception fırlatıyoruz
    # - Yanlış deneylerin önüne geçer
    raise ValueError("act 'relu' veya 'silu' olmalı.")


class ChannelAttentionFusionT(nn.Module):
    def __init__(
        self,
        channels: int,
        reduction: int = 16,
        min_hidden: int = 4,
        fusion: str = "softmax",
        gate: str = "sigmoid",
        temperature: float = 0.9,
        learnable_temperature: bool = False,
        eps: float = 1e-6,
        act: str = "relu",
        bias: bool = True,
        fusion_router_hidden: int = 16,
        return_fusion_weights: bool = False,
        t_min: float = 0.5,
        t_max: float = 3.0,
        router_temperature: float = 1.5,
        beta_ca: float = 0.35,
    ):
        super().__init__()

        if channels < 1:
            raise ValueError("channels >= 1 olmalı.")
        if reduction < 1:
            raise ValueError("reduction >= 1 olmalı.")
        if fusion not in ("sum", "softmax"):
            raise ValueError("fusion 'sum' veya 'softmax' olmalı.")
        if temperature <= 0:
            raise ValueError("temperature pozitif olmalı.")
        if fusion == "softmax" and fusion_router_hidden < 1:
            raise ValueError("fusion_router_hidden >= 1 olmalı.")
        if t_min <= 0 or t_max <= 0 or t_min > t_max:
            raise ValueError("T clamp aralığı hatalı.")
        if router_temperature <= 0:
            raise ValueError("router_temperature pozitif olmalı.")
        if beta_ca < 0:
            raise ValueError("beta_ca >= 0 olmalı.")

        # Küçük sayısal kararlılık sabiti (softplus/log/ bölme vb. işlemlerde patlamayı önler)
        self.eps = float(eps)

        # AvgPool + MaxPool bilgisini nasıl birleştireceğimizi belirler (örn: "sum" / "softmax" gibi)
        self.fusion = fusion

        # Debug/analiz için router'ın (fusion) ağırlıklarını dışarı döndürmek istersek True
        self.return_fusion_weights = return_fusion_weights

        # Kanal maskesinin son gate fonksiyonu (sigmoid veya hardsigmoid) seçilir
        self.gate_fn = _get_gate(gate)

        # Temperature clamp sınırları: CA'nın sertliğini kontrol eder (T bu aralıkta tutulur)
        self.t_min = float(t_min)
        self.t_max = float(t_max)

        # Router (avg/max birleştirme) tarafında softmax kullanılıyorsa onun sıcaklığı (softmax keskinliği)
        self.Tr = float(router_temperature)

        # CA çıktısını yumuşatma katsayısı (maskeyi 1'e doğru çeker; agresifliği azaltır)
        self.beta_ca = float(beta_ca)

        # CA içindeki "MLP" ara kanal sayısı (reduction ile düşürür, ama min_hidden altına inmez)
        hidden = max(int(min_hidden), int(channels) // int(reduction))

        # Global Average Pooling: (B,C,H,W) -> (B,C,1,1) kanal özetini çıkarır
        self.avg_pool = nn.AdaptiveAvgPool2d(1)

        # Global Max Pooling: (B,C,H,W) -> (B,C,1,1) en güçlü aktivasyonları yakalar
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        # 1x1 conv "fc1": kanalları hidden boyutuna indirir (SE/CBAM tarzı bottleneck)
        self.fc1 = nn.Conv2d(channels, hidden, kernel_size=1, bias=bias)

        # Ara aktivasyon (relu veya silu)
        self.act = _get_act(act)

        # 1x1 conv "fc2": hidden'dan tekrar channels'a çıkarır (kanal maskesi logits üretimi)
        self.fc2 = nn.Conv2d(hidden, channels, kernel_size=1, bias=bias)

        if self.fusion == "softmax":
            # Softmax fusion: avg/max bilgisinden öğrenilebilir şekilde fusion ağırlıkları üretmek için küçük bir router.
            # Giriş 2*channels: (avg_pool, max_pool) concat edildiği varsayımıyla.
            self.fusion_router = nn.Sequential(
                nn.Conv2d(2 * channels, fusion_router_hidden, kernel_size=1, bias=bias),
                nn.ReLU(inplace=True),
                nn.Conv2d(hidden, channels, kernel_size=1, bias=bias),
            )
            # Son katmanı sıfırdan başlatıyoruz:
            # Başlangıçta logits ~ 0 olsun -> softmax tarafsız başlasın (genelde 0.5/0.5 gibi),
            # erken eğitimde agresif yönlenme / over-suppression riskini azaltır.
            last = self.fusion_router[-1]
            nn.init.zeros_(last.weight)
            if last.bias is not None:
                nn.init.zeros_(last.bias)
        else:
            # fusion="sum" gibi modlarda router'a gerek yok (sabit birleştirme).
            self.fusion_router = None

        self.learnable_temperature = bool(learnable_temperature)

        if self.learnable_temperature:
            # İsteğe bağlı strict kontrol (istersen aç)
            # if not (self.t_min <= float(temperature) <= self.t_max):
            #     raise ValueError(f"temperature [{self.t_min}, {self.t_max}] aralığında olmalı (learnable_temperature=True).")
            # Stabil başlangıç: temperature'ı clamp edip t_raw'ı onunla başlat.
            t0 = float(temperature)
            # Sınırların tam üstüne yapışmayı engellemek için küçük güvenlik payı
            lo = self.t_min + self.eps
            hi = self.t_max - self.eps
            # Eğer kullanıcı ters aralık verdiyse zaten yukarıda kontrol var; yine de defansif:
            if lo >= hi:
                lo = self.t_min
                hi = self.t_max
            t0 = min(max(t0, lo), hi)
            t_inv = softplus_inverse(torch.tensor(t0), eps=self.eps)
            self.t_raw = nn.Parameter(t_inv)
        else:
            # Fixed temperature için de clamp'li başlatmak istersen (opsiyonel):
            # t0 = float(temperature)
            # t0 = min(max(t0, self.t_min), self.t_max)
            # self.register_buffer("T", torch.tensor(t0))
            self.register_buffer("T", torch.tensor(float(temperature)))

    def get_T(self, x: torch.Tensor) -> torch.Tensor:
            if self.learnable_temperature:
                T = F.softplus(self.t_raw) + self.eps
            else:
                T = self.T
            T = T.to(device=x.device, dtype=x.dtype)
            T_clamped = T.clamp(self.t_min, self.t_max)
            # Opsiyonel: clamp'e yapışma kontrolü (debug)
            # if self.training and getattr(self, "monitor_T", False):
            #     with torch.no_grad():
            #         self._T_value = float(T_clamped.detach().cpu())
            #         self._T_hit_min = bool((T_clamped <= (self.t_min + 1e-6)).item())
            #         self._T_hit_max = bool((T_clamped >= (self.t_max - 1e-6)).item())
            return T_clamped
    
    def mlp(self,s:torch.Tensor) -> torch.Tensor:
        return self.fc2(self.act(self.fc1(s)))
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # (B,C,H,W) -> (B,C,1,1): kanal başına global özet (ortalama aktivasyon)
        avg_s = self.avg_pool(x)
        # (B,C,H,W) -> (B,C,1,1): kanal başına global özet (en güçlü aktivasyon)
        max_s = self.max_pool(x)
        # Avg özetini MLP'den geçir: (B,C,1,1) -> (B,C,1,1) kanal logits/score üretir
        a = self.mlp(avg_s)
        # Max özetini MLP'den geçir: (B,C,1,1) -> (B,C,1,1) kanal logits/score üretir
        m = self.mlp(max_s)
        # İsteğe bağlı debug çıktısı: fusion ağırlıkları (sum modunda yok)
        self.fusion_w = None
        if self.fusion == "sum":
            # Basit birleştirme: Avg ve Max katkılarını eşit kabul et
            z = a + m
        else: # Her sample için “avg mı daha güvenilir, max mı daha güvenilir?”
            # Router tabanlı birleştirme: Avg/Max için ağırlık öğren
            # Not: concat edilen şey genelde (a,m) veya (avg_s,max_s) olabilir;
            # biz burada (avg_s,max_s) ile router'a "ham özet" veriyoruz.
            s_cat = torch.cat([avg_s, max_s], dim=1)  # (B,2C,1,1)
            # Router logits üretir: (B,2C,1,1) -> (B,2,1,1) veya benzeri; flatten ile (B,2)
            logits = self.fusion_router(s_cat).flatten(1)
            # logits → router’ın ham kararı
            # / Tr → bu kararın yumuşatılması / keskinleştirilmesi
            # softmax → avg vs max ağırlıkları
            # Temperature (Tr): softmax kararının keskinliğini ayarlar
            # Tr, fusion router’ın avg–max kararını ne kadar keskin vereceğini kontrol eden sıcaklıktır.
            # Tr küçük -> daha keskin (biri baskın), Tr büyük -> daha yumuşak (dengeli)
            fusion_w = torch.softmax(logits / self.Tr, dim=1) # (B,2) # Bu şu demek: B tane satır var, her satırda 2 tane sayı var.
            # Avg ve Max için ağırlıkları (B,1,1,1) şekline getir (broadcast için)
            w0 = fusion_w[:, 0].view(-1, 1, 1, 1)  # avg ağırlığı
            # : → “satırların hepsini al”
            # 0 → “sadece 0. sütunu al”
            # Yani: “Bütün satırlardan, sadece ilk elemanı seç.”
            w1 = fusion_w[:, 1].view(-1, 1, 1, 1)  # max ağırlığı
            # Ağırlıklı birleşim: z = w_avg * a + w_max * m
            z = w0 * a + w1 * m
        # Temperature'ı al: learnable/sabit olabilir, device/dtype uyumlu ve clamp'li gelir
        T = self.get_T(x)
        # Kanal attention maskesi:
        # z/T ile "sertlik" ayarlanır; ardından gate (sigmoid/hardsigmoid) ile (0,1) aralığına sıkıştırılır
        ca = self.gate_fn(z / T)
        # CA'yı yumuşatma (beta_ca):
        # ca doğrudan x'e vurulmaz; 1 etrafında karıştırılır -> agresif bastırmayı azaltır
        # beta_ca=0  => scale_ca = 1 (CA etkisi kapalı)
        # beta_ca=1  => scale_ca = ca (CA tam uygulanır)
        scale_ca = 1.0 + self.beta_ca * (ca - 1.0)
        # Feature'ı kanal ölçeğiyle yeniden ağırlıklandır
        y = x * scale_ca
        # İstenirse debug amaçlı fusion ağırlıklarını da döndür
        # (fusion="sum" ise fusion_w None olabilir)
        if self.return_fusion_weights and (fusion_w is not None):
            return y, ca, fusion_w
        # Default dönüş: çıktı ve kanal maskesi
        return y, ca

----
----
----

# 2) Büyük resim şeması ( Coordinate Attention Plus )

**Bu modül, feature map’i iki ayrı eksende özetliyor: biri “yükseklik profili” (H boyunca), diğeri “genişlik profili” (W boyunca). Her profil hem mean hem max ile çıkarılıyor (daha sağlam istatistik). Sonra her eksende hem local depthwise hem dilated depthwise ile çok-ölçekli (multi-scale) bilgi toplanıyor, 1x1 mixer ile kanallar karıştırılıyor. H ve W çıktıları tek bir “shared bottleneck”te birleştirilip işleniyor (C→mid). Ardından tekrar H ve W olarak split edilip iki ayrı head ile attn_h ve attn_w maskeleri üretiliyor. Maskeler direkt vurulmuyor; alpha ile 1’e karıştırılıyor (yumuşak attention), sonra h ve w çarpılıp global beta ile tekrar 1’e yaklaştırılıyor ve scale clamp ile güvenlik bariyerine alınarak x ile çarpılıyor. İstersen en sonda opsiyonel spatial gate ile ekstra bir uzamsal gate daha uygulanıyor.**
```bash
x (B,C,H,W)
   |
   |-- H-profile (W üzerinden özet) ------------------------------\
   |        h_profile = 0.5*(mean over W + max over W)            |
   |        => (B,C,H,1)                                          |
   |                                                              |
   |-- W-profile (H üzerinden özet) ---------------------------\  |
            w_profile = 0.5*(mean over H + max over H)          | |
            => (B,C,1,W)                                        | |
                                                                | |
           (Multi-scale yönsel conv: local DW + dilated DW)     | |
                |                                               | |
      h_local_dw(3x1) + h_dilated_dw(3x1,d)                     | |
                |                                               | |
           h_channel_mixer (1x1)                                | |
                |                                               | |
              h_ms (B,C,H,1)                                    | |
                                                                | |
                                  w_local_dw(1x3) + w_dilated_dw(1x3,d)
                                                   |
                                            w_channel_mixer (1x1)
                                                   |
                                           w_ms (B,C,1,W)
                                                   |
                                     permute -> (B,C,W,1) -------/
                                                   |
                         concat along "height axis" (dim=2)
                         hw = cat([h_ms, w_ms_perm]) -> (B,C,H+W,1)
                                                   |
                                   shared bottleneck (C -> mid)
                      proj1x1 + norm + act  -> refine1x1 + norm + act
                                                   |
                                      mid (B,mid,H+W,1)
                                                   |
                    split back into H and W parts (dim=2)
                     mid_h: (B,mid,H,1)      mid_w: (B,mid,W,1)
                                                   |
                               permute mid_w -> (B,mid,1,W)
                                                   |
                       head convs (mid -> C) + hardsigmoid gate
                attn_h = head_h(mid_h) -> (B,C,H,1)  in (0,1)
                attn_w = head_w(mid_w) -> (B,C,1,W)  in (0,1)
                                                   |
                          alpha ile 1’e karıştırma (yumuşatma)
           alpha_h = sigmoid(alpha_h_raw)  (scalar)   alpha_w = sigmoid(alpha_w_raw)
           scale_h = (1-alpha_h) + alpha_h*attn_h     -> (B,C,H,1)
           scale_w = (1-alpha_w) + alpha_w*attn_w     -> (B,C,1,W)
                                                   |
                             birleşik ölçek (broadcast ile çarpılır)
                       scale = scale_h * scale_w -> (B,C,H,W)
                                                   |
                        global beta ile ekstra yumuşatma
                       scale = 1 + beta*(scale-1)
                                                   |
                          clamp güvenlik bariyeri
                    scale = clamp(scale_min, scale_max)
                                                   |
                                                   v
                                   out = x * scale  (B,C,H,W)
                                                   |
                           (opsiyonel) spatial gate (ek kapı)
                 sg = DW(3x3) + PW(1x1) -> hardsigmoid -> 1’e karıştır
                           out = out * sg



In [ ]:
class HSwish(nn.Module):
    def forward(self, x: torch.Tensor):
        # HSwish (Hard-Swish) aktivasyonu:
        # - Swish'e benzer ama daha ucuzdur (relu6 ile parçalı-lineer)
        # - YOLO / mobil CNN'lerde sık kullanılır (hız + stabil gradient)
        # - Formül: x * relu6(x + 3) / 6
        return x * F.relu6(x + 3.0, inplace=True) / 6.0


def make_norm(norm: str, ch: int):
    # Normalizasyon seçici:
    # - "bn"   : BatchNorm2d (batch istatistikleri; küçük batch'te oynayabilir)
    # - "gn"   : GroupNorm (batch bağımsız; detection/küçük batch'te daha stabil)
    # - "none" : Identity (norm kapalı)
    norm = norm.lower()
    if norm == "bn":
        return nn.BatchNorm2d(ch)

    if norm == "gn":
        # GN: num_groups (g) kanalları tam bölmeli -> ch % g == 0
        # g'yi 32'den başlatıp bölünebilir hale gelene kadar düşürüyoruz.
        g = min(32, ch)
        while ch % g != 0 and g > 2:
            g //= 2
        # Hâlâ bölünmüyorsa fallback:
        # - ch çiftse g=2 (iki grup)
        # - ch tekse g=1 (tek grup, LN benzeri)
        if ch % g != 0:
            g = 2 if (ch % 2 == 0) else 1
        return nn.GroupNorm(g, ch)
    if norm == "none":
        return nn.Identity()
    raise ValueError("norm 'none', 'bn', 'gn' dışında olamaz.")


class CoordinateAttPlus(nn.Module):
    def __init__(
        self,
        in_channels: int,          # Giriş kanal sayısı (C)
        reduction: int = 32,       # Bottleneck oranı: C -> mid (yaklaşık C/reduction)
        min_mid_channels: int = 8, # mid için alt sınır (çok küçülmesin)
        act: str = "hswish",       # Aktivasyon: "hswish" / "relu" / "silu"
        init_alpha: float = 0.7,   # Başlangıç alpha hedefi (sigmoid sonrası ~0.7)
        learnable_alpha: bool = True, # alpha öğrenilebilir mi?
        beta: float = 0.35,        # Global yumuşatma: scale'i 1'e çeker (agresifliği azaltır)
        dilation: int = 2,         # Dilated DW conv dilation (receptive field büyütür)
        norm: str = "gn",          # Norm seçimi: bn/gn/none
        use_spatial_gate: bool = False,  # Ek spatial gate aç/kapat
        spatial_gate_beta: float = 0.35, # Spatial gate yumuşatma katsayısı
        scale_min: float = 0.6,    # Final scale clamp alt sınırı (en fazla bastırma)
        scale_max: float = 1.6,    # Final scale clamp üst sınırı (en fazla güçlendirme)
        head_init_std: float = 0.01, # Head init std: maskeler başlangıçta sakin kalsın
    ):
        super().__init__()
        # -------------------------
        # Parametre validasyonları (fail-fast)
        # -------------------------
        if in_channels < 1:
            raise ValueError("in_channels >= 1 olmalı.")
        if reduction < 1:
            raise ValueError("reduction >= 1 olmalı.")
        if dilation < 1:
            raise ValueError("dilation >= 1 olmalı.")
        if scale_min <= 0 or scale_max <= 0 or scale_min > scale_max:
            raise ValueError("scale clamp aralığı hatalı.")
        if head_init_std <= 0:
            raise ValueError("head_init_std pozitif olmalı.")
        # Global yumuşatma ve clamp sınırları (guardrail)
        self.beta = float(beta)
        self.scale_min = float(scale_min)
        self.scale_max = float(scale_max)
        # -------------------------
        # Bottleneck kanal hesabı (mid)
        # -------------------------
        # mid: squeeze boyutu; çok küçülürse bilgi kaybı artar.
        # mid_floor: in_channels'a göre "alt limit" koyuyoruz (8..32 arası, yaklaşık C/4)
        mid_floor = max(8, min(32, int(in_channels) // 4))
        mid = max(int(min_mid_channels), int(in_channels) // int(reduction))
        mid = max(mid, int(mid_floor))  # en az mid_floor olsun
        # -------------------------
        # Aktivasyon seçimi
        # -------------------------
        act_l = act.lower()
        if act_l == "hswish":
            self.act = HSwish()
        elif act_l == "relu":
            self.act = nn.ReLU(inplace=True)
        elif act_l == "silu":
            self.act = nn.SiLU(inplace=True)
        else:
            raise ValueError("act 'hswish', 'relu', 'silu' olmalı.")
        # -------------------------
        # Shared bottleneck: h ve w yolları ortak bir bottleneck'te işleniyor
        # -------------------------
        # (C -> mid) projeksiyon + norm
        self.shared_bottleneck_proj = nn.Conv2d(in_channels, mid, 1, bias=False)
        self.shared_bottleneck_norm = make_norm(norm, mid)
        # mid içinde bir refine (1x1) daha + norm
        self.shared_bottleneck_refine = nn.Conv2d(mid, mid, 1, bias=False)
        self.shared_bottleneck_refine_norm = make_norm(norm, mid)
        # -------------------------
        # Yönsel (H/W) çok-ölçekli DW konvlar
        # -------------------------
        # Local DW: küçük receptive field (yakın komşuluk)
        self.h_local_dw = nn.Conv2d(
            in_channels, in_channels, kernel_size=(3, 1), padding=(1, 0),
            groups=in_channels, bias=False
        )
        self.w_local_dw = nn.Conv2d(
            in_channels, in_channels, kernel_size=(1, 3), padding=(0, 1),
            groups=in_channels, bias=False
        )
        # Dilated DW: daha geniş receptive field (uzak bağlam)
        # padding=(d,0)/(0,d) seçimi, stride=1 iken boyutu korumak için:
        # effective_kernel = (k-1)*d + 1 -> k=3, d=2 => 5; padding=2 ile same korunur.
        d = int(dilation)
        self.h_dilated_dw = nn.Conv2d(
            in_channels, in_channels, kernel_size=(3, 1), padding=(d, 0),
            dilation=(d, 1), groups=in_channels, bias=False
        )
        self.w_dilated_dw = nn.Conv2d(
            in_channels, in_channels, kernel_size=(1, 3), padding=(0, d),
            dilation=(1, d), groups=in_channels, bias=False
        )
        # -------------------------
        # Kanal karıştırma (1x1): DW çıktılarında kanallar arası etkileşim yok;
        # 1x1 ile kanalları tekrar karıştırıyoruz.
        # -------------------------
        self.h_channel_mixer = nn.Conv2d(in_channels, in_channels, 1, bias=True)
        self.w_channel_mixer = nn.Conv2d(in_channels, in_channels, 1, bias=True)
        # -------------------------
        # Attention head'leri: mid -> C maskeleri üretir (h ve w ayrı)
        # -------------------------
        self.h_attention_head = nn.Conv2d(mid, in_channels, 1, bias=True)
        self.w_attention_head = nn.Conv2d(mid, in_channels, 1, bias=True)

        # Head init: maskeler eğitim başında agresifleşmesin (0.5 civarı yumuşak başlasın)
        nn.init.normal_(self.h_attention_head.weight, mean=0.0, std=float(head_init_std))
        nn.init.normal_(self.w_attention_head.weight, mean=0.0, std=float(head_init_std))
        if self.h_attention_head.bias is not None:
            nn.init.zeros_(self.h_attention_head.bias)
        if self.w_attention_head.bias is not None:
            nn.init.zeros_(self.w_attention_head.bias)

        # -------------------------
        # Alpha init: 1 ile attention'ı karıştırma gücü
        # alpha_raw logit uzayında tutulur; forward'da sigmoid ile (0,1)'e gelir.
        # -------------------------
        eps = 1e-6
        a0 = float(init_alpha)
        a0 = min(max(a0, eps), 1.0 - eps)  # logit(0/1) -> inf olmasın
        raw0 = torch.logit(torch.tensor(a0), eps=eps)

        if learnable_alpha:
            # Öğrenilebilir alpha: model eğitimde maskeyi ne kadar uygulatacağını ayarlar
            self.alpha_h_raw = nn.Parameter(raw0.clone())
            self.alpha_w_raw = nn.Parameter(raw0.clone())
        else:
            # Sabit alpha: state'e girsin, device ile taşınsın diye buffer
            self.register_buffer("alpha_h_raw", raw0.clone())
            self.register_buffer("alpha_w_raw", raw0.clone())
        # -------------------------
        # Opsiyonel spatial gate: ekstra bir 3x3 DW + 1x1 ile uzamsal gate
        # (İstersen aç; detection’da over-suppression riskine dikkat.)
        # -------------------------
        self.use_spatial_gate = bool(use_spatial_gate)
        self.spatial_gate_beta = float(spatial_gate_beta)
        if self.use_spatial_gate:
            self.spatial_gate_dw = nn.Conv2d(
                in_channels, in_channels, 3, padding=1, groups=in_channels, bias=False
            )
            self.spatial_gate_pw = nn.Conv2d(in_channels, in_channels, 1, bias=True)
        # Debug için son maskeleri saklamak (stats almak için)
        self._last_ah = None
        self._last_aw = None

    def forward(self, x: torch.Tensor):
        # x: (B, C, H, W)
        _, _, H, W = x.shape
        # -------------------------
        # 1) Coordinate profile çıkarma
        # -------------------------
        # h_profile: W boyunca özet -> (B, C, H, 1)
        # w_profile: H boyunca özet -> (B, C, 1, W)
        # mean + max karışımı: hem genel seviye hem de güçlü aktivasyonlar taşınır
        h_profile = 0.5 * (x.mean(dim=3, keepdim=True) + x.amax(dim=3, keepdim=True))
        w_profile = 0.5 * (x.mean(dim=2, keepdim=True) + x.amax(dim=2, keepdim=True))
        # -------------------------
        # 2) Multi-scale yönsel konv (local + dilated) + channel mixer
        # -------------------------
        h_ms = self.h_channel_mixer(self.h_local_dw(h_profile) + self.h_dilated_dw(h_profile))
        w_ms = self.w_channel_mixer(self.w_local_dw(w_profile) + self.w_dilated_dw(w_profile))
        # w_ms: (B,C,1,W) -> concat için (B,C,W,1) gibi hizalama
        w_ms = w_ms.permute(0, 1, 3, 2)
        # h_ms: (B,C,H,1), w_ms: (B,C,W,1) -> dim=2 boyunca birleştir -> (B,C,H+W,1)
        hw = torch.cat([h_ms, w_ms], dim=2)
        # -------------------------
        # 3) Shared bottleneck ile ortak işleme (C -> mid)
        # -------------------------
        mid = self.act(self.shared_bottleneck_norm(self.shared_bottleneck_proj(hw)))
        mid = self.act(self.shared_bottleneck_refine_norm(self.shared_bottleneck_refine(mid)))
        # -------------------------
        # 4) mid'i tekrar H ve W parçalarına ayır
        # -------------------------
        mid_h, mid_w = torch.split(mid, [H, W], dim=2)
        mid_w = mid_w.permute(0, 1, 3, 2)  # (B,mid,W,1) -> (B,mid,1,W)
        # -------------------------
        # 5) Head'lerle maskeleri üret (0..1)
        # -------------------------
        attn_h = F.hardsigmoid(self.h_attention_head(mid_h), inplace=False)  # (B,C,H,1)
        attn_w = F.hardsigmoid(self.w_attention_head(mid_w), inplace=False)  # (B,C,1,W)
        # Debug için sakla
        self._last_ah = attn_h.detach()
        self._last_aw = attn_w.detach()
        # -------------------------
        # 6) Alpha ile 1'e karıştırma (yumuşak uygula)
        # -------------------------
        alpha_h = torch.sigmoid(self.alpha_h_raw).to(device=x.device, dtype=x.dtype)  # scalar
        alpha_w = torch.sigmoid(self.alpha_w_raw).to(device=x.device, dtype=x.dtype)  # scalar

        scale_h = (1.0 - alpha_h) + alpha_h * attn_h   # 1 ile attn_h arasında karışım
        scale_w = (1.0 - alpha_w) + alpha_w * attn_w   # 1 ile attn_w arasında karışım
        # -------------------------
        # 7) Global beta yumuşatma + clamp guardrail
        # -------------------------
        scale = scale_h * scale_w
        scale = 1.0 + self.beta * (scale - 1.0)                # scale'i 1'e çek
        scale = scale.clamp(self.scale_min, self.scale_max)    # aşırı bastırma/boost engeli
        # Uygula
        out = x * scale
        # -------------------------
        # 8) Opsiyonel spatial gate
        # -------------------------
        if self.use_spatial_gate:
            sg = self.spatial_gate_pw(self.spatial_gate_dw(x))
            sg = F.hardsigmoid(sg, inplace=False)
            sg = 1.0 + self.spatial_gate_beta * (sg - 1.0)
            out = out * sg

        return out

    @torch.no_grad()
    def last_mask_stats(self):
        # Debug: son attn_h/attn_w maskelerinin basit istatistikleri
        if (self._last_ah is None) or (self._last_aw is None):
            return None
        ah = self._last_ah
        aw = self._last_aw
        return {
            "a_h": {"min": float(ah.min()), "mean": float(ah.mean()), "max": float(ah.max()), "std": float(ah.std())},
            "a_w": {"min": float(aw.min()), "mean": float(aw.mean()), "max": float(aw.max()), "std": float(aw.std())},
        }

# 3-) Büyük Resim Şeması

```bash
Girdi: x  (B,C,H,W)
  |
  |------------------------------------|
  |                                    |
  |     [ChannelAttentionFusionT]      |
  |   (CBAM Channel + T + beta_ca)     |
  |                                    |
  |  y_ca, ca_map, (fusion_w?) = CA(x) |
  |        (B,C,H,W)    (B,C,1,1)      |
  |                                    |
  |------------------------------------|
                  |
                  v
     [CoordinateAttPlus]  (CoordAtt + alpha_h/alpha_w + beta + clamp + opsiyonel spatial gate)
                  |
                  v
              y = Coord(y_ca)          (B,C,H,W)
                  |
                  v
         if residual == False:
              return y  (veya debug ise maps/statlar)
                  |
                  v
      residual == True  ->  Residual Mixer
                  |
                  v
        alpha = sigmoid(alpha_raw)      (0..1)   (device/dtype uyumlu)
        alpha_eff = alpha               (başlangıç)
                  |
                  v
      if training AND monitor:
          x_std    = std_per_sample(x)
          y_std    = std_per_sample(y)
          out_tmp  = x + alpha*(y-x)        (residual karışımın ham hali)
          out_std  = std_per_sample(out_tmp)

          r_block  = clamp(y_std / x_std, 0..10)
          r_out    = clamp(out_std / x_std, 0..10)

          r_ema <- EMA update(r_out)
          alpha_eff <- compute_alpha_eff(alpha, r_ema, r_min, rescue_mode)
          (monitor_stats doldur)
                  |
                  v
        out = x + alpha_eff * (y - x)      (B,C,H,W)
                  |
                  v
      return out
      (+ return_maps ise: ca_map, fusion_w, coord_stats, monitor_stats da döner)
```
**Not: fusion_w sadece CA tarafında ca_fusion="softmax" ve return_maps=True iken anlamlı.**

* Bu blok bir feature map alıyor. Önce Channel Attention çalışıyor. Burada model şuna bakıyor: “Hangi kanallar önemli?” Bunun için her kanalın ortalamasını ve maksimumunu alıyor, küçük bir ağdan geçiriyor ve her kanal için bir önem değeri üretiyor. Bu değerler direkt kullanılmıyor, 1’e doğru yumuşatılıyor ki feature map bir anda ölmesin. Sonuçta x, kanal bazında hafifçe güçlenmiş ya da bastırılmış oluyor.

* Sonra bu çıktı Coordinate Attention’a giriyor. Burada model şuna bakıyor: “Bu bilgi daha çok dikey yönde mi önemli, yatay yönde mi?” Bunun için yükseklik ve genişlik yönlerinde ayrı ayrı özetler çıkarıyor, hem yakın çevreyi hem de biraz daha geniş alanı gören konvolüsyonlardan geçiriyor. Buradan iki maske çıkıyor: biri H yönü, biri W yönü için. Bu maskeler de yine yumuşak şekilde uygulanıyor, aşırı bastırma engelleniyor.

* Eğer residual kapalıysa, bu noktada çıkan feature map direkt çıktı oluyor.

* Residual açıksa, blok x ile attention’dan geçmiş y arasında karışım yapıyor. Yani tamamen y’ye atlamıyor, x’ten y’ye doğru kontrollü bir adım atıyor. Bu adımın büyüklüğünü alpha belirliyor.

* Eğitim sırasında izleme açıksa, blok kendi kendini kontrol ediyor: “Attention’dan sonra feature map çok mu düzleşti?” diye bakıyor. Bunu standart sapma ile ölçüyor. Eğer feature fazla bastırılmışsa, alpha’yı otomatik olarak küçültüyor. Böylece blok kendini yumuşatıyor ama tamamen kapanmıyor.

**Sonuçta bu yapı, attention uygular ama feature’ı öldürmez. Hem kanal bazında, hem yön bazında bakar, üstüne bir de kendini denetleyip gerektiğinde geri adım atar.**

In [ ]:
class CBAMChannelPlusCoord(nn.Module):
    def __init__(
        self,
        channels: int,                 # Giriş/çıkış kanal sayısı (C). Blok drop-in olacağı için sabit kalır.
        # -------------------------
        # Channel Attention (CA / CBAM channel) parametreleri
        # -------------------------
        ca_reduction: int = 16,         # CA içindeki bottleneck oranı: C -> hidden (≈ C/ca_reduction)
        ca_min_hidden: int = 4,         # hidden alt sınırı: C küçükse MLP tamamen “çökmesin”
        ca_fusion: str = "softmax",     # AvgPool+MaxPool nasıl birleşecek: "sum" sabit, "softmax" learnable router
        ca_gate: str = "sigmoid",       # Kanal maskesi gate fonksiyonu: sigmoid / hardsigmoid
        ca_temperature: float = 0.9,    # CA temperature başlangıç değeri (sertlik kontrolü); learnable olabilir
        ca_act: str = "relu",           # CA MLP aktivasyonu (relu/silu gibi)
        ca_fusion_router_hidden: int = 16,  # softmax fusion router ara kanal sayısı (küçük MLP)
        learnable_temperature: bool = False, # CA temperature öğrenilsin mi? (True: t_raw parametre; False: buffer)
        ca_t_min: float = 0.5,               # CA temperature clamp alt sınırı (T çok küçülüp hard gate olmasın)
        ca_t_max: float = 3.0,               # CA temperature clamp üst sınırı (T çok büyüyüp etkisizleşmesin)
        ca_router_temperature: float = 1.5,  # Fusion router softmax sıcaklığı (avg vs max karar keskinliği)
        beta_ca: float = 0.35,               # CA çıktısını 1'e yaklaştırma (agresifliği yumuşatma): scale=1+beta*(ca-1)
        # -------------------------
        # Coordinate Attention (CoordAttPlus) parametreleri
        # -------------------------
        coord_reduction: int = 32,       # Coord bottleneck oranı: C -> mid
        coord_min_mid: int = 8,          # mid alt sınırı
        coord_act: str = "hswish",       # Coord içindeki aktivasyon (hswish/relu/silu)
        coord_init_alpha: float = 0.7,   # Coord alpha başlangıcı (1'e karıştırma gücü; başlangıçta attention ne kadar devrede)
        coord_learnable_alpha: bool = True,  # Coord alpha öğrenilsin mi? (h ve w için ayrı raw parametre)
        coord_beta: float = 0.35,        # Coord global yumuşatma: scale=1+beta*(scale-1)
        coord_dilation: int = 2,         # Coord dilated DW conv dilation (receptive field büyütür)
        coord_norm: str = "gn",          # Coord norm tipi (bn/gn/none). Küçük batch'te GN daha stabil.
        coord_use_spatial_gate: bool = False,   # Opsiyonel ek spatial gate (ek bir “kapı” daha)
        coord_spatial_gate_beta: float = 0.35,  # Spatial gate yumuşatma katsayısı (1'e karıştırma)
        coord_scale_min: float = 0.6,    # Coord final scale clamp alt sınırı (en fazla bastırma)
        coord_scale_max: float = 1.6,    # Coord final scale clamp üst sınırı (en fazla güçlendirme)
        coord_head_init_std: float = 0.01, # Coord head init std (maskeler başta 0/1'e yapışmasın)
        # -------------------------
        # Blok seviyesi Residual + Monitor/Rescue parametreleri
        # -------------------------
        residual: bool = True,           # True: out = x + alpha_eff*(y-x) (enerji kurtarma / stabilite)
        alpha_init: float = 0.75,        # Residual karışım başlangıcı (sigmoid sonrası hedef). Bloğun gücü.
        learnable_alpha: bool = False,   # Residual alpha öğrenilsin mi? (True: parametre; False: buffer)
        monitor: bool = False,           # Training’de over-suppression izleme aç/kapat (std oranı ölçümü)
        r_min: float = 0.45,             # Kabul edilen minimum enerji oranı eşiği (r_out < r_min => rescue devreye girer)
        ema_momentum: float = 0.95,      # r_ema güncelleme momentumu (yüksek = daha yavaş ama daha stabil)
        min_rescue_ratio: float = 0.2,   # ratio_floor modunda ratio için alt taban (tamamen sıfırlamasın diye)
        alpha_eff_min: float = 0.2,      # alpha_floor modunda alpha_eff alt sınırı (bloğu tamamen kapatmamak için)
        rescue_mode: str = "ratio_floor",# Rescue stratejisi ("ratio_floor" / "alpha_floor" gibi)
        return_maps: bool = False,       # Debug modu: ca_map, fusion_w, coord_stats, monitor_stats döndür
    ):
        super().__init__()

        self.return_maps = bool(return_maps)
        self.residual = bool(residual)

        self.monitor = bool(monitor)
        self.r_min = float(r_min)
        self.ema_m = float(ema_momentum)
        self.min_rescue_ratio = float(min_rescue_ratio)
        self.alpha_eff_min = float(alpha_eff_min)
        self.rescue_mode = str(rescue_mode)

        self.ca = ChannelAttentionFusionT(
            channels=channels,
            reduction=ca_reduction,
            min_hidden=ca_min_hidden,
            fusion=ca_fusion,
            gate=ca_gate,
            temperature=ca_temperature,
            learnable_temperature=learnable_temperature,
            eps=1e-6,
            act=ca_act,
            bias=True,
            fusion_router_hidden=ca_fusion_router_hidden,
            return_fusion_weights=self.return_maps,
            t_min=ca_t_min,
            t_max=ca_t_max,
            router_temperature=ca_router_temperature,
            beta_ca=beta_ca,
        )

        self.coord = CoordinateAttPlus(
            in_channels=channels,
            reduction=coord_reduction,
            min_mid_channels=coord_min_mid,
            act=coord_act,
            init_alpha=coord_init_alpha,
            learnable_alpha=coord_learnable_alpha,
            beta=coord_beta,
            dilation=coord_dilation,
            norm=coord_norm,
            use_spatial_gate=coord_use_spatial_gate,
            spatial_gate_beta=coord_spatial_gate_beta,
            scale_min=coord_scale_min,
            scale_max=coord_scale_max,
            head_init_std=coord_head_init_std,
        )

        if residual:
            #Amaç aynı — kısıtlı bir değeri (alpha veya T) güvenli ve öğrenilebilir yapmak; 
            # bunun için alpha’da sigmoid–logit, temperature’da softplus–inverse kullanıyoruz.
            eps = 1e-6
            a0 = float(alpha_init) 
            # alpha_init bizim verdiğimiz başlangıç “karışım gücü”. Ama bu değer 0 veya 1 olursa problem çıkıyo
            # alpha = 0 demek: residual karışım tamamen kapalı → out = x
            # alpha = 1 demek: residual karışım full açık → out = y
            # İşte bu yüzden “tam 0” veya “tam 1” değerlerini yasaklıyoruz.
            # alpha_init = 1 verdin → a0 = 1-eps olur. Yani: sonsuzluklardan kaçış / sayısal guardrail.
            a0 = min(max(a0,eps),1.0-eps)
            raw0 = torch.logit(torch.tensor(a0),eps=eps)
            # sigmoid(z) = a ise
            # logit(a) = z
            # “Ben başlangıçta alpha = a0 istiyorum → o zaman alpha_raw = logit(a0) olmalı.”
            if learnable_alpha:
                self.alpha_raw = nn.Parameter(raw0) # nn.Parameter(raw0) → optimizer bunu günceller → alpha öğrenilir.
            else:
                self.register_buffer("alpha_raw",raw0)
        self.register_buffer("r_ema",torch.tensor(1.0))

    def alpha(self,x:torch.tensor) -> torch.Tensor:
        # Bu fonksiyonun amacı residual karışım katsayısı olan alphayı 
        # “her durumda güvenli ve uyumlu” şekilde üretmek.
        # Residual karışım için alphayı (0–1) aralığında tutup, 
        # YOLO/AMP’de patlamasın diye x ile aynı cihaz/tipte döndürüyor.
        if (not self.residual) or (not hasattr(self,"alpha_raw")): # Residual yoksa veya alpha_raw yoksa
            # “Alpha’ya ihtiyaç yok” demek.
            # 1.0 döndürerek varsayılan güvenli davranış veriyor.
            return x.new_tensor(1.0)
        return torch.sigmoid(self.alpha_raw).to(device=x.device,dtype=x.dtype) # alpha_raw logit uzayında tutuluyor.
        # sigmoid(alpha_raw) ile alpha’yı (0,1) aralığına kilitliyor.

    @staticmethod
    def _std_per_sample(x: torch.Tensor) -> torch.Tensor:
        # Bu fonksiyon "her örnek (sample) için" standart sapmayı hesaplayıp,
        # sonra batch içindeki örneklerin ortalamasını döndürür.
        # Amaç: monitor/rescue mantığında "x'in aktivasyon yayılımı (std)" gibi bir büyüklük ölçmek.
        # Yani: tensor ne kadar dalgalı/kontrastlı, aktivasyonlar ne kadar saçılmış?
        # x.float():
            # - x'in dtype'ı fp16/bf16 olabilir.
            # - std gibi istatistiksel hesaplar düşük hassasiyette daha gürültülü/instabil olur.
            # - O yüzden float32'ye çeviriyoruz (daha stabil numerik hesap).
        x_float = x.float()
        # flatten(1):
            # - x genelde (B, C, H, W) gibi bir tensordur.
            # - flatten(1) demek: batch boyutunu (B) koru, geri kalan her şeyi tek vektöre düzleştir.
            # - Sonuç shape: (B, C*H*W)
            # - Böylece her sample için tek bir uzun vektör elde ederiz.
        x_flat = x_float.flatten(1)
        # std(dim=1, unbiased=False):
            # - dim=1: her sample'ın (C*H*W) vektörü üzerinde std hesapla → sonuç shape (B,)
            # - unbiased=False (correction=0):
            #   * "popülasyon std" gibi hesaplar; N-1 düzeltmesi yapmaz.
            #   * N çok küçükse (özellikle N=1 gibi saçma uç durumlarda) unbiased=True std NaN üretebilir.
            #   * unbiased=False bu NaN riskini ciddi azaltır ve daha stabil davranır.
        per_sample_std = x_flat.std(dim=1, unbiased=False)  # shape: (B,)
        # mean():
            # - batch içindeki tüm örneklerin std değerlerini ortalar
            # - tek skaler döner: "batch'in ortalama std'si"
        return per_sample_std.mean()

    @torch.no_grad()
    def _update_r_ema(self, r_out: torch.Tensor):
        # Bu fonksiyon r_out’u filtreleyip (yumuşatıp) r_ema’ya yazar ki rescue mekanizması her adım kafayı yemesin.
        # Her forward’da ölçtüğün r_out değerini zıplamasın diye yumuşatıp r_ema içine yazmak.
        # r_out, attention’dan SONRA feature’ların ne kadar “canlı kaldığını” gösteren tek bir sayıdır.
        r_det = r_out.detach().to(device=self.r_ema.device, dtype=self.r_ema.dtype)
        self.r_ema.mul_(self.ema_m).add_((1.0 - self.ema_m) * r_det)
        # mul_(x) :: “Kendini x ile çarp, yeni tensor üretme.” 
            # a = a * 0.9  ::  ama yeni a oluşturmaz, aynı a değişir
        # add_(x) :: “Kendine x ekle, yeni tensor üretme.” 
            # a = a + b     # ama yine aynı a değişir
     
    ## KESİNLİKLE DİKKAT 
    # Bu fonksiyon, r_ema düşükse (feature map fazla “ölmüşse”) alphayı otomatik azaltıp bloğu
            #  yumuşatır;
    #  ama min_rescue_ratio sayesinde alpha’nın tamamen sıfırlanmasına izin vermez.
    def compute_alpha_eff(self,x:torch.Tensor , alpha:torch.Tensor) -> torch.Tensor:
        # attention fazla bastırıyorsa alpha’yı otomatik kısmak
        # İçeride yapılan iş aslında tek mekanizma: alpha’yı, ratio diye bir güvenlik katsayısıyla çarpıp küçültmek.
        # r_ema: Son çıktının “enerji oranı” EMA’sı. (genelde out_std / x_std)
        # r_min: “Benim kabul ettiğim minimum enerji oranı” eşiği.
        ratio = (self.r_ema.detach() / max(self.r_min,1e-12)).clamp(0.0,1.0)
        # Eğer r_ema >= r_min ise :: r_ema / r_min >= 1 olur → clamp ile 1.0’a çekilir → kısma yok
        # Eğer r_ema < r_min ise  :: oran 1’den küçük olur → clamp ile 0–1 arası kalır → alpha küçülür → blok yumuşar
        ratio = ratio.to(device=x.device,dtype=x.dtype)
        # AMP/FP16 vs için. alpha * ratio yaparken mismatch yemeyesin.Bunu istemiyoruz.
        if self.rescue_mode == "ratio_floor":
            ratio = ratio.clamp(self.min_rescue_ratio,1.0)
            # ratio çok küçülürse (ör. 0.01), alpha neredeyse sıfırlanır → blok “tam kapanır”
            # min_rescue_ratio ile “en az şu kadar açık kalsın” diyoruz. :: Yani rescue çalışsa bile alpha tamamen ölmez.
            return alpha * ratio
        alpha_eff = alpha * ratio
        # Bu modda önce çarpıp sonra clamp ediyorsun.
        return alpha_eff.clamp(self.min_rescue_ratio , 1.0)
    
    def forward(self, x: torch.Tensor):
        monitor_stats = None

        if self.return_maps:
            y_ca, ca_map, fusion_w = self.ca(x)
            y = self.coord(y_ca)

            if not self.residual:
                coord_stats = self.coord.last_mask_stats()
                return y, ca_map, fusion_w, coord_stats, None

            alpha = self._alpha(x)
            alpha_eff = alpha

            if self.training and self.monitor:
                x_std = self._std_per_sample(x)
                y_std = self._std_per_sample(y)

                out_tmp = x + alpha * (y - x)
                out_std = self._std_per_sample(out_tmp)

                r_block = (y_std / (x_std + 1e-12)).clamp(0.0, 10.0)
                r_out = (out_std / (x_std + 1e-12)).clamp(0.0, 10.0)

                self._update_r_ema(r_out)
                alpha_eff = self._compute_alpha_eff(x, alpha)

                monitor_stats = {
                    "x_std": float(x_std.detach()),
                    "y_std": float(y_std.detach()),
                    "out_std_pre": float(out_std.detach()),
                    "r_block": float(r_block.detach()),
                    "r_out_pre": float(r_out.detach()),
                    "r_ema": float(self.r_ema.detach()),
                    "alpha": float(alpha.detach()),
                    "alpha_eff": float(alpha_eff.detach()),
                    "rescue_mode": self.rescue_mode,
                }

            out = x + alpha_eff * (y - x)
            coord_stats = self.coord.last_mask_stats()
            return out, ca_map, fusion_w, coord_stats, monitor_stats

        y_ca, _ = self.ca(x)
        y = self.coord(y_ca)

        if not self.residual:
            return y

        alpha = self._alpha(x)
        alpha_eff = alpha

        if self.training and self.monitor:
            x_std = self._std_per_sample(x)
            out_tmp = x + alpha * (y - x)
            out_std = self._std_per_sample(out_tmp)
            r_out = (out_std / (x_std + 1e-12)).clamp(0.0, 10.0)

            self._update_r_ema(r_out)
            alpha_eff = self._compute_alpha_eff(x, alpha)

        out = x + alpha_eff * (y - x)
        return out